In [3]:
# Import required libraries
import pandas as pd
import dash
from dash import html
# import dash_html_components as html
from dash import dcc
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0, max=10000,
                                    step=1000,
                                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                    value=[min_payload, max_payload]
                                ),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
   
    if entered_site == 'ALL':
        data = spacex_df.groupby(['Launch Site'], as_index=False)['class'].sum()
        data['class'] = data['class']/data['class'].sum()
        fig = px.pie(data, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Count the occurrences of each class
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        
        # Create the pie chart
        fig = px.pie(class_counts, names='class', values='count', 
                     title=f'Total Success Launches for {entered_site}',
                     labels={'class': 'Outcome', 'count': 'Count'},
                     # color='class', color_discrete_map={0: 'red', 1: 'green'}
                    )
        
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])

def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range 
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)] 
    if entered_site == 'ALL': 
        fig = px.scatter(filtered_df, 
                         x='Payload Mass (kg)', 
                         y='class', 
                         color="Booster Version", 
                         title='Correlation between Payload and Mission Outcome for All Sites', 
                         labels={'class': 'Launch Outcome'}, 
                         hover_data=['Launch Site']) 
    else: 
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site] 
        fig = px.scatter(filtered_df, 
                         x='Payload Mass (kg)', y='class', 
                         color="Booster Version", 
                         title=f'Correlation between Payload and Mission Outcome for {entered_site}', 
                         labels={'class': 'Launch Outcome'}, 
                         hover_data=['Launch Site']) 
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8052)


### Analysis with Your Completed Dashboard

Now that your dashboard is up and running, you can use it to analyze the SpaceX launch data effectively. Here’s how you can answer the questions you mentioned:

1. **Which site has the largest successful launches?**
   - Look at the pie chart for total successful launches by site. The site with the largest pie segment represents the one with the most successful launches.

2. **Which site has the highest launch success rate?**
   - This can be determined by calculating the ratio of successful launches to the total launches for each site. You may need to further analyze the data using pandas.

3. **Which payload range(s) has the highest launch success rate?**
   - Use the scatter plot and adjust the payload range using the slider. Observe the success rates for different ranges to identify which payload ranges have the highest success rates.

4. **Which payload range(s) has the lowest launch success rate?**
   - Similarly, use the scatter plot to find the payload ranges with the lowest success rates by adjusting the payload slider and observing the launch outcomes.

5. **Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?**
   - Color-label the scatter plot points by Booster Version. Filter the data by booster version and calculate the success rates to determine which version has the highest success rate.

Here are some code snippets to help with these analyses:

#### Site with the Largest Successful Launches:
```python
successful_launches = spacex_df[spacex_df['class'] == 1]
site_counts = successful_launches['Launch Site'].value_counts()
largest_success_site = site_counts.idxmax()
print("Site with the largest successful launches:", largest_success_site)
```

#### Site with the Highest Launch Success Rate:
```python
site_success_rate = spacex_df.groupby('Launch Site')['class'].mean()
highest_success_rate_site = site_success_rate.idxmax()
print("Site with the highest launch success rate:", highest_success_rate_site)
```

#### Payload Range with Highest and Lowest Launch Success Rates:
```python
def payload_success_rate(min_payload, max_payload):
    payload_range_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & 
                                 (spacex_df['Payload Mass (kg)'] <= max_payload)]
    return payload_range_df['class'].mean()

# Example usage:
print("Success rate for payload range 0-1000 kg:", payload_success_rate(0, 1000))
```

#### F9 Booster Version with Highest Launch Success Rate:
```python
booster_success_rate = spacex_df.groupby('Booster Version')['class'].mean()
highest_success_rate_booster = booster_success_rate.idxmax()
print("F9 Booster version with the highest launch success rate:", highest_success_rate_booster)
```

By utilizing these analyses along with your dashboard, you should be able to extract meaningful insights from the SpaceX launch data.

In [42]:
successful_launches = spacex_df[spacex_df['class'] == 1]
site_counts = successful_launches['Launch Site'].value_counts()
largest_success_site = site_counts.idxmax()
print("Site with the largest successful launches:", largest_success_site)

site_success_rate = spacex_df.groupby('Launch Site')['class'].mean()
highest_success_rate_site = site_success_rate.idxmax()
print("Site with the highest launch success rate:", highest_success_rate_site)

def payload_success_rate(min_payload, max_payload):
    payload_range_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & 
                                 (spacex_df['Payload Mass (kg)'] <= max_payload)]
    return payload_range_df['class'].mean()

# Example usage:
print("Success rate for payload range 0-1000 kg:", payload_success_rate(0, 1000))

booster_success_rate = spacex_df.groupby('Booster Version')['class'].mean()
highest_success_rate_booster = booster_success_rate.idxmax()
print("F9 Booster version with the highest launch success rate:", highest_success_rate_booster)


Site with the largest successful launches: KSC LC-39A
Site with the highest launch success rate: KSC LC-39A
Success rate for payload range 0-1000 kg: 0.2
F9 Booster version with the highest launch success rate: F9 B4 B1039.1
